### **В этом блокноте мы конвертируем записи из списков блокадников (простой текст) в табличку, извлекая из них информацию регулярками и костылями.**

In [3]:
import os
import re
from bs4 import BeautifulSoup as beat_soup
import pandas as pd
from tqdm import tqdm

In [4]:
PATH_TO_HTMLS = "./blockade_base_new1/" # путь к папке с хтмлками
DEATH_DATE = r"смерти: [1,2]?[0-9]? ?\w{3,10} 19[0-9]{2}" # смертельная регулярка

Достаём пути ко всем хтмлкам.

In [5]:
htmls = []

for _, dirs, files in os.walk(PATH_TO_HTMLS):
    for file in files:
        htmls.append(PATH_TO_HTMLS + file)  

Готовим функции для извлечения текста из хтмл и извлечения информации из собственно текста.

In [7]:
def get_text(html_path: str) -> str:
    """ достаем текстовую статью с хтмл-странички """
    with open(html_path, "r") as f:    
        contents = f.read() 
        soup = beat_soup(contents, 'lxml')
        
    # странички сделаны странно и теги у них не всегда одинаковы - пробуем оба варианта
    if soup.find("div", class_ = "col-sm-12"):
        return soup.find("div", class_ = "col-sm-12").text.strip().replace("\n", "")
    return soup.find("div", class_ = "col-sm-9").text.strip().replace("\n", "")

In [8]:
def get_person_info(text: str) -> dict:
    """ создаем словарь с информацией о человеке """
    person = text 
    info = {"fio": "no_info", "birth_date": "no_info",
            "address": "no_info", "death_date": "no_info", "burial_place": "no_info",
            "source": "no_info"}
    
    fio = person.split(",")[0] # чаще всего отделено запятой, но иногда нет... :
    if "Место" in fio:
        fio = fio.split("Место")[0].strip()
    if "Дата" in fio:
        fio = fio.split("Дата")[0].strip() 
    
    try:
        birth_date = re.findall(r"[0-3]?[0-9]?.?[0-1]?[0-9]?.?1[8-9][0-9]{2}", 
                                person.split("г.р.")[0].split("род")[1])[0].strip("., ")  
        # (да. выше было что-то уродливое. но оно... типа... работает)
    except:
        birth_date = "no_info" # иногда даты рождения не бывает
        
    info["fio"], info["birth_date"] = fio, birth_date
    
    # дергаем место проживания, если есть
    if "Место проживания" in person:  
        addr = person.split("Место проживания: ")[1] 
        if "Дата смерти" in person:
            address = addr.split("Дата")[0].strip(". ")
        else:
            address = addr.split("(")[0].strip(". ")
            info["address"] = address
     
    # дергаем всякое про смерть, если есть (чаще нет)
    if "Дата смерти" in person: 
        if re.findall(DEATH_DATE, person):
            death_date = re.findall(DEATH_DATE, person)[0].replace("смерти: ", "")
            info["death_date"] = death_date 
            
    if "Место захоронения" in person:
        burial_place = person.split("Место захоронения: ")[-1].split("(")[0]
        info["burial_place"] = burial_place
    
    # источник достается не всегда -- иногда на страничке он просто в другом блоке
    # это надо дописать
    if "(" in person:
        source = person.split("(")[-1].strip(") ")
        info["source"] = source
    
    return info

Обходим все хтмлки и собираем информацию для каждой из них + добавляем айдишники.

In [9]:
dicts = []

for x in tqdm(htmls):    
    try: # была какая-то странная ошибка с одним из файлов, закостылила
        x_info = get_person_info(get_text(x))
        x_info["id"] = x.split("/")[-1].strip(".html")
    except:
        print (x) 
    dicts.append(x_info)    

  3%|▎         | 1116/35186 [00:08<05:03, 112.29it/s]

./blockade_base_new1/1806053.html


  4%|▎         | 1305/35186 [00:09<04:08, 136.15it/s]

./blockade_base_new1/1661713.html


 10%|█         | 3635/35186 [00:27<04:16, 123.24it/s]

...

./blockade_base_new1/1973274.html


 94%|█████████▎| 32984/35186 [04:55<00:17, 126.05it/s]

./blockade_base_new1/1972176.html


100%|██████████| 35186/35186 [05:13<00:00, 112.06it/s]

./blockade_base_new1/workspace.xml
./blockade_base_new1/blockade_base_new1.iml
./blockade_base_new1/modules.xml
./blockade_base_new1/misc.xml
./blockade_base_new1/R_User_Library.xml


Собираем табличку.

In [10]:
blockade_table = pd.DataFrame(columns = dicts[0].keys())

In [11]:
for n in blockade_table: 
    blockade_table[n] = [x[n] for x in dicts]

In [12]:
blockade_table

,fio,birth_date,address,death_date,burial_place,source,id
0,Хоникова Юлия Терентьевна,05.12.1930,"г. Санкт-Петербург, Калининский р-н",no_info,no_info,"Они пережили Блокаду, т. 13",1919776
1,Иванова Валентина Михайловна,20.01.1921,Кировский р-н,no_info,no_info,"Они пережили Блокаду, т. 4",1955406
2,Козлова Нина Алексеевна,07.01.1926,Выборгский р-н,no_info,no_info,"Они пережили Блокаду, т. 5",1672808
3,Соболева Мария Куприяновна,24.12.1917,"г. Санкт-Петербург, Калининский р-н",no_info,no_info,"Они пережили Блокаду, т. 11",1812623
4,Яковлев Борис Георгиевич,03.03.1937,"г. Санкт-Петербург, Выборгский р-н",no_info,no_info,"Они пережили Блокаду, т. 14",1941354
5,Полканова Агафья Васильевна,1903,"г. Санкт-Петербург, ул. Большая Зеленина, д. 2...",no_info,no_info,"Они пережили Блокаду, т. 9",1768943
6,Попова Роза Ефимовна,24.01.1933,"г. Санкт-Петербург, Фрунзенский р-н",no_info,no_info,"Они пережили Блокаду, т. 9",1771619
7,Горобинский Юрий Львович,17.08.1938,Кировский р-н,no_info,no_info,"Они пережили Блокаду, т. 3",1563606
8,Чебуранов Федор Андреевич,10.12.1927,"г. Санкт-Петербург, ул. Новосивковская, д. 15,...",no_info,no_info,"Они пережили Блокаду, т. 13",1933207
9,Колдашева Ольга Григорьевна,13.04.1928,Фрунзенский р-н,no_info,no_info,"Они пережили Блокаду, т. 5",1673894


In [10]:
# выгружаем в файлик
blockade_table.to_csv("blockade_example.csv", sep = "\t")